### Import libraries

In [0]:
# import data and packages
import json
import requests
import os, sys, time

# import computing packages
import pandas as pd
import numpy as ny

# plotting
import plotly.graph_objects as go

# Setup pandas display options
pd.options.display.max_rows = 100
pd.options.display.max_columns = 200

### Custom Functions

In [0]:
def download_data(url):
    df = pd.read_csv(url)
    return df

def keep_states_only(us_state_abbrev, state):
  # check if the state name is valid, and if yes return the state abbrev code
  if state in us_state_abbrev.keys():
    return us_state_abbrev[state]
  else:
    return None

### State Code Lookup Table

In [0]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'Washington DC':'DC'
}

### Download Boston Scientic Donation Data

In [4]:
# Define google sheets url for download
sheet_uuid= '1xr6SKzLMqnMvdTggzra4ov90q2DMNo-CtiZli72HWWY'

# Use string comprehension to replace %s with the above sheet_uuid
# this url below converts google sheet to csv
url = ('https://docs.google.com/spreadsheet/ccc?key=%s&output=csv' \
  % (sheet_uuid) ) 
print ('Downloand URL for dataframe import: %s' % url) 

# convert url to the dataframe
df = download_data(url)

# Visualize the data
df.head(5)

Downloand URL for dataframe import: https://docs.google.com/spreadsheet/ccc?key=1xr6SKzLMqnMvdTggzra4ov90q2DMNo-CtiZli72HWWY&output=csv


,State,Face Shield
0,California,"117,900"
1,Puerto Rico,"50,000"
2,Michigan,"50,000"
3,Texas,"59,000"
4,Minnesota,"45,000"


### Processing the data

In [5]:
# Sort values by Face Shields
df.sort_values(by=['Face Shield'], ascending=False, inplace=True)

# Map to state code
df['state_code']= df.apply(
    lambda row: (keep_states_only(us_state_abbrev, row['State'])),axis=1
)

df.dropna(inplace=True)
df.nunique()

# Remove all commas
df = df.replace(',','', regex=True)

# print breakdown
df.nunique()

State          29
Face Shield    21
state_code     29
dtype: int64

In [6]:
# Create bar plot graph of the face shield deliveries between states
x=df['state_code']
y=df['Face Shield']

# TODO: Figure out how to plot this in descending order by quantity of 'Face Shield'


fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            textposition='auto',
        )])
fig.update_layout(
    title='Face Shields Produced+Donated by Boston Scientific - April 26th, 2020', xaxis={'categoryorder':'total descending'}
)

fig.show()

### Download state populations

In [7]:
# Define google sheets url for download
sheet_uuid= '1bTl8A0hdLN0z-xvXYRwlpZ_X3kTa619riFn4PYMTjJ0'

# Use string comprehension to replace %s with the above sheet_uuid
# this url below converts google sheet to csv
url = ('https://docs.google.com/spreadsheet/ccc?key=%s&output=csv' \
  % (sheet_uuid) ) 
print ('Downloand URL for dataframe import: %s' % url) 

# convert url to the dataframe
pop_df = download_data(url)

# Visualize the data
pop_df.nunique()

Downloand URL for dataframe import: https://docs.google.com/spreadsheet/ccc?key=1bTl8A0hdLN0z-xvXYRwlpZ_X3kTa619riFn4PYMTjJ0&output=csv


State         57
Population    57
dtype: int64

In [0]:
pop_df['state_code']=pop_df.apply(
    lambda row: (keep_states_only(us_state_abbrev, row['State'])),axis=1
)

pop_df.dropna(inplace=True)

### Add empty state columns to boston sci df
TODO: Make idempotent (i.e. run twice without causing error)

In [9]:
# Inner Join
df = pop_df[['state_code','State','Population']].join(
    df[['state_code','Face Shield']].set_index('state_code'),
    on='state_code',  how='left')

# Remove all commas
df = df.replace(',','', regex=True)

# Fill NAs
df.fillna(0,inplace=True)

# Add text column
df['text'] = df['State'].astype(str) + '<br>' + \
    'Face Shields: ' + df['Face Shield'].astype(str) + '<br>'

df.nunique()

state_code     52
State          52
Population     52
Face Shield    22
text           52
dtype: int64

### Combine Boston Scientific Data into pop_df - Create merged_df

In [10]:
# Inner Join
merged_df = pop_df.join(
    df[['state_code','Face Shield']].set_index('state_code'),
    on='state_code',  how='left')

# Remove all commas
merged_df = merged_df.replace(',','', regex=True)

# Fill NAs
merged_df.fillna(0,inplace=True)

# Convert to numeric values
merged_df[['Face Shield','Population']] = merged_df[['Face Shield','Population']].astype(int)

# get normalized face shield counts
merged_df['normalized_face_shield']=merged_df.apply(
    lambda row: (row['Face Shield'] / row['Population']),axis=1
)
merged_df['normalized_face_shield_per_100k']=merged_df.apply(
    lambda row: (row['Face Shield'] / row['Population'] * 100000),axis=1
)

# round out the normalized columns
merged_df['normalized_face_shield'] = merged_df['normalized_face_shield'].round(6)
merged_df['normalized_face_shield_per_100k'] = merged_df['normalized_face_shield_per_100k'].round(1)

# Add text column
merged_df['text'] = merged_df['State'].astype(str) + '<br>' + \
    'Face Shields Per 100k: ' + merged_df['normalized_face_shield_per_100k'].astype(str) + '<br><br>' + \
    'Population: ' + merged_df['Population'].astype(str) + '<br>' + \
    'Face Shields: ' + merged_df['Face Shield'].astype(str) + '<br>'

# print example out
print (len(merged_df.index))
merged_df.head(3)



52


,State,Population,state_code,Face Shield,normalized_face_shield,normalized_face_shield_per_100k,text
5,Alabama,4903185,AL,0,0.000000,0.0,Alabama<br>Face Shields Per 100k: 0.0<br><br>P...
6,Alaska,731545,AK,0,0.000000,0.0,Alaska<br>Face Shields Per 100k: 0.0<br><br>Po...
7,Arizona,7278717,AZ,5000,0.000687,68.7,Arizona<br>Face Shields Per 100k: 68.7<br><br>...


### Map Raw Face Shield Donations

In [11]:
# Make the figure
fig = go.Figure(data=go.Choropleth(
    locations=df['state_code'], # Spatial coordinates
    z = df['Face Shield'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'blues',
    text=df['text'], # hover text
    colorbar_title = "Face Shields >",
    hoverinfo='text',
    marker_line_color='Grey', # line markers between states
    marker_opacity=0.8, 
    zmin=0,
    zmax=50000,
    marker_line_width=0.5,
))

fig.update_layout(
    title_text='Face Shields Produced+Donated by Boston Scientific - April 26th, 2020',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)

fig.show()

### Map the normalized face shields

In [12]:
# Make the figure
fig = go.Figure(data=go.Choropleth(
    locations=merged_df['state_code'], # Spatial coordinates
    z = merged_df['normalized_face_shield_per_100k'], # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'blues',
    text=merged_df['text'], # hover text
    colorbar_title = ">Shields Per 100k",
    hoverinfo='text',
    marker_line_color='Grey', # line markers between states
    marker_opacity=0.8, 
    zmin=0,
    zmax=500,
    marker_line_width=0.5,
))

fig.update_layout(
    title_text='Face Shields Produced+Donated by Boston Scientific (Normalized to Population) - April 26th, 2020',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)

fig.show()

### Download the figure

In [0]:
# Download the figure
go.Figure.write_html(fig, file='N95s Delivered by HHS by April 3rd - 2020.html',
                     config={'responsive': True}, include_plotlyjs='cdn')

### GOAL: Shields per covid case

In [0]:
### Download NYTimes State level data 

In [0]:
def download_nytimes_data(url, date):
    covid_df = pd.read_csv(url)
    covid_df = covid_df.loc[covid_df['date'] == date]
    return covid_df

In [16]:
date = "2020-04-25"
url = "https://github.com/nytimes/covid-19-data/raw/master/us-states.csv"

covid_df =  download_nytimes_data(url, date)
covid_df.nunique()

date       1
state     55
fips      55
cases     55
deaths    52
dtype: int64

### Process NYtimes data

In [17]:
# Sort values by Face Shields
covid_df.sort_values(by=['cases'], ascending=False, inplace=True)

covid_df['state_code']=covid_df.apply(
    lambda row: (keep_states_only(us_state_abbrev, row['state'])),axis=1
)

# Remove all commas
covid_df = covid_df.replace(',','', regex=True)

# dropna
covid_df.dropna(inplace=True)

# print breakdown
covid_df.nunique()

date           1
state         55
fips          55
cases         55
deaths        52
state_code    55
dtype: int64

In [0]:
# TODO: Put all the others in per state
covid_df['text'] = covid_df['state'].astype(str) + '<br>' + \
    'Cases: ' + covid_df['cases'].astype(str) + '<br>' + \
    'Deaths: ' + covid_df['deaths'].astype(str) + '<br>'

### Make the Covid state map

In [19]:
# Make the figure
fig = go.Figure(data=go.Choropleth(
    locations=covid_df['state_code'], # Spatial coordinates
    z = covid_df['cases'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'reds',
    text=covid_df['text'], # hover text
    colorbar_title = ">COVID19 Caes",
    hoverinfo='text',
    marker_line_color='Black', # line markers between states
    marker_opacity=0.8, 
    zmin=0,
    zmax=100000,
    marker_line_width=0.5,
))

fig.update_layout(
    title_text='COVID19 Cases by State - April 26th, 2020',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)

fig.show()

### Merge nytimes data into boston scientific and population information

In [20]:
# Inner Join
merged_full_df = merged_df.join(
    covid_df[['state_code','cases','deaths']].set_index('state_code'),
    on='state_code',  how='left')

# Remove all commas
merged_full_df = merged_full_df.replace(',','', regex=True)

# Fill NAs
merged_full_df.fillna(0,inplace=True)

# Convert to numeric values
merged_full_df[['cases','deaths']] = merged_full_df[
    ['cases','deaths']].astype(int)

# get normalized cases counts
merged_full_df['normalized_cases']=merged_full_df.apply(
    lambda row: (row['cases'] / row['Population']),axis=1
)
merged_full_df['normalized_cases_per_100k']=merged_full_df.apply(
    lambda row: (row['cases'] / row['Population'] * 100000),axis=1
)

# KEY LOGIC
# Shields donated per covid case
merged_full_df['shields_donated_per_covid_case']=merged_full_df.apply(
    lambda row: (row['Face Shield'] / row['cases']),axis=1
)

# round out the normalized columns
merged_full_df['normalized_cases'] = merged_full_df['normalized_cases'].round(6)
merged_full_df['normalized_cases_per_100k'] = merged_full_df['normalized_cases_per_100k'].round(1)



# Add text column
merged_full_df['text'] = merged_full_df['State'].astype(str) + '<br>' + \
    'Face Shields Per Covid Case: ' + merged_full_df['shields_donated_per_covid_case'].astype(str) + '<br><br>' + \
    'COVID19 Cases: ' + merged_full_df['cases'].astype(str) + '<br>' + \
    'COVID19 Death: ' + merged_full_df['deaths'].astype(str) + '<br>' + \
    'Face Shields: ' + merged_full_df['Face Shield'].astype(str) + '<br>' + \
    'Population: ' + merged_full_df['Population'].astype(str)

# Add text column for nomalized Covid19 Cases
merged_full_df['text2'] = merged_full_df['State'].astype(str) + '<br>' + \
    'COVID19 Cases - Normalzied to Population: ' + merged_full_df['normalized_cases_per_100k'].astype(str) + '<br><br>' + \
    'COVID19 Cases: ' + merged_full_df['cases'].astype(str) + '<br>' + \
    'COVID19 Death: ' + merged_full_df['deaths'].astype(str) + '<br>' + \
    'Face Shields: ' + merged_full_df['Face Shield'].astype(str) + '<br>' + \
    'Face Shields Per Covid Case: ' + merged_full_df['shields_donated_per_covid_case'].astype(str) + '<br>' + \
    'Population: ' + merged_full_df['Population'].astype(str)
    

# print example out
print (len(merged_full_df.index))
merged_full_df.head(3)

52


,State,Population,state_code,Face Shield,normalized_face_shield,normalized_face_shield_per_100k,text,cases,deaths,normalized_cases,normalized_cases_per_100k,shields_donated_per_covid_case,text2
5,Alabama,4903185,AL,0,0.000000,0.0,Alabama<br>Face Shields Per Covid Case: 0.0<br...,6213,213,0.001267,126.7,0.000000,Alabama<br>COVID19 Cases - Normalzied to Popul...
6,Alaska,731545,AK,0,0.000000,0.0,Alaska<br>Face Shields Per Covid Case: 0.0<br>...,337,7,0.000461,46.1,0.000000,Alaska<br>COVID19 Cases - Normalzied to Popula...
7,Arizona,7278717,AZ,5000,0.000687,68.7,Arizona<br>Face Shields Per Covid Case: 0.7961...,6280,275,0.000863,86.3,0.796178,Arizona<br>COVID19 Cases - Normalzied to Popul...


### Plot of normalized cases per 100k people

In [21]:
fig = go.Figure(data=go.Choropleth(
    locations=merged_full_df['state_code'], # Spatial coordinates
    z = merged_full_df['normalized_cases_per_100k'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'reds',
    text=merged_full_df['text2'], # hover text
    colorbar_title = "COVID19 cases per 100k",
    hoverinfo='text',
    marker_line_color='Black', # line markers between states
    marker_opacity=0.8, 
    zmin=0,
    zmax=1000,
    marker_line_width=0.5,
))

fig.update_layout(
    title_text='COVID19 Cases by State (Normalized to Population) - April 26th, 2020',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)

fig.show()

### Plot of face shield need (via shields donated per covid19 case)

In [22]:
fig = go.Figure(data=go.Choropleth(
    locations=merged_full_df['state_code'], # Spatial coordinates
    z = merged_full_df['shields_donated_per_covid_case'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'viridis',
    text=merged_full_df['text'], # hover text
    colorbar_title = "Shields Per COVID19 Case",
    hoverinfo='text',
    marker_line_color='Black', # line markers between states
    marker_opacity=0.8, 
    zmin=0,
    zmax=5,
    marker_line_width=0.5,
    reversescale=True
))

fig.update_layout(
    title_text='Shields Donated Per COVID19 Case - April 26th, 2020',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(255, 255, 255)'),
)

fig.show()

### Create priority list for remaining Boston Scientific Face Shields

In [23]:
# select columns of interest
output_df = merged_full_df[[
    'State',
    'Face Shield',
    'normalized_cases_per_100k',
    'shields_donated_per_covid_case']]	
output_df = output_df.rename(columns={
    'Face Shield':'Face Shields Donated',
    'normalized_cases_per_100k':'COVID19 Cases Per 100K Population',
    'shields_donated_per_covid_case': 'Shields Donated Per COVID19 Case'
})

output_df.sort_values(by=['Shields Donated Per COVID19 Case', 
    'COVID19 Cases Per 100K Population'], ascending=[True,False],inplace=True)

output_df.head(55)

,State,Face Shields Donated,COVID19 Cases Per 100K Population,Shields Donated Per COVID19 Case
11,Connecticut,0,689.5,0.000000
46,South Dakota,0,242.7,0.000000
29,Mississippi,0,192.1,0.000000
20,Iowa,0,161.9,0.000000
33,Nevada,0,147.4,0.000000
50,Vermont,0,135.1,0.000000
34,New Hampshire,0,131.9,0.000000
36,New Mexico,0,127.0,0.000000
5,Alabama,0,126.7,0.000000
49,Utah,0,123.2,0.000000


### Create a map with all traces

In [24]:
fig = go.Figure()

# First trace
fig = fig.add_trace(go.Choropleth(
    locations=df['state_code'], # Spatial coordinates
    z = df['Face Shield'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'blues',
    text=df['text'], # hover text
    colorbar_title = "Face Shields >",
    hoverinfo='text',
    marker_line_color='Grey', # line markers between states
    marker_opacity=0.8, 
    zmin=0,
    zmax=50000,
    marker_line_width=0.5,
))

# Second trace
fig = fig.add_trace(go.Choropleth(
    locations=merged_full_df['state_code'], # Spatial coordinates
    z = merged_full_df['normalized_cases_per_100k'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'reds',
    text=merged_full_df['text2'], # hover text
    colorbar_title = "COVID19 cases per 100k",
    hoverinfo='text',
    marker_line_color='Black', # line markers between states
    marker_opacity=0.8, 
    zmin=0,
    zmax=1000,
    marker_line_width=0.5,
    visible=False
))

# Third trace
fig = fig.add_trace(go.Choropleth(
    locations=merged_full_df['state_code'], # Spatial coordinates
    z = merged_full_df['shields_donated_per_covid_case'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'RdBu',
    text=merged_full_df['text'], # hover text
    colorbar_title = "Shields Per COVID19 Case",
    hoverinfo='text',
    marker_line_color='Black', # line markers between states
    marker_opacity=0.8, 
    zmin=0,
    zmax=2,
    marker_line_width=0.5,
    visible=False
))


# Create button list
buttons_list=list([
    dict(label='Face Shields', method="update",
          args=[{"visible": ([True,False,False])},
                {"title": 'Boston Scientific - 700K Face Shields Produced for Donation - 4/26/20'}]),
    dict(label='COVID19 Cases', method="update",
          args=[{"visible": ([False,True,False])},
                {"title": "COVID19 Cases Normalized to State Population - 4/26/20"}]),
    dict(label='Shields Per COVID19 Case', method="update",
          args=[{"visible": ([False,False,True])},
                {"title": "Boston Scientific - Face Shields Produced Per COVID19 Case - 4/26/20"}]),
    

    ])

# Add buttons
fig.update_layout(
    updatemenus=list([
        dict(
            type = "buttons",
            active=0,
            showactive=True,
            direction="right",
            pad={"l":75,"t": -40},
            yanchor="top",
            xanchor="left",
            buttons=buttons_list,
        )])
)

# Update layout
fig.update_layout(
  title_text='Boston Scientific - 700K Face Shields Produced for Donation - 4/26/20',
  geo = dict(
      scope='usa',
      projection=go.layout.geo.Projection(type = 'albers usa'),
      showlakes=True, # lakes
      lakecolor='rgb(255, 255, 255)')
)

# show figure
fig.show()

# Download the figure From Sunny Mui
go.Figure.write_html(fig,
    file='index_boston_scientic_face_shields_covid19_cases_normalized.html', 
    config={'responsive': True}, include_plotlyjs='cdn')